In [1]:
# load environment variables
from dotenv import load_dotenv
_ = load_dotenv(dotenv_path="../.env")

In [2]:
# Get document from secinsights.ai production API (avoids having to seed db each time running this notebook)
import requests
base_url = "https://llama-app-backend.onrender.com"     # Production backend base URL
document_id = "922f4a9f-7bc9-4fb1-b9b8-fa5a84c1cbcc"    # META 10-K 2022

response = requests.get(f"{base_url}/api/document/{document_id}")
data = response.json()
data

{'id': '922f4a9f-7bc9-4fb1-b9b8-fa5a84c1cbcc',
 'created_at': '2023-11-03T22:42:50.306199',
 'updated_at': '2023-11-03T22:42:50.306199',
 'url': 'https://d687lz8k56fia.cloudfront.net/sec-edgar-filings/0001326801/10-K/0001326801-23-000013/primary-document.pdf',
 'metadata_map': {'sec_document': {'cik': '0001326801',
   'year': 2022,
   'doc_type': '10-K',
   'company_name': 'Meta Platforms, Inc.',
   'company_ticker': 'META',
   'accession_number': '0001326801-23-000013',
   'filed_as_of_date': '2023-02-02T00:00:00',
   'date_as_of_change': '2023-02-01T00:00:00',
   'period_of_report_date': '2022-12-31T00:00:00'}}}

In [3]:
from app import schema
document = schema.Document(**data)
document

Document(id=UUID('922f4a9f-7bc9-4fb1-b9b8-fa5a84c1cbcc'), created_at=datetime.datetime(2023, 11, 3, 22, 42, 50, 306199), updated_at=datetime.datetime(2023, 11, 3, 22, 42, 50, 306199), url='https://d687lz8k56fia.cloudfront.net/sec-edgar-filings/0001326801/10-K/0001326801-23-000013/primary-document.pdf', metadata_map={<DocumentMetadataKeysEnum.SEC_DOCUMENT: 'sec_document'>: {'cik': '0001326801', 'year': 2022, 'doc_type': '10-K', 'company_name': 'Meta Platforms, Inc.', 'company_ticker': 'META', 'accession_number': '0001326801-23-000013', 'filed_as_of_date': '2023-02-02T00:00:00', 'date_as_of_change': '2023-02-01T00:00:00', 'period_of_report_date': '2022-12-31T00:00:00'}})

In [4]:
# from eval import get_dummy_doc
# document = get_dummy_doc()
# document

In [5]:
from app.chat.engine import fetch_and_read_document
document = fetch_and_read_document(document)     # merge document pages into a single document

LOADING DOCUMENT WITH SimpleDirectoryReader: META 10-K 2022
MERGING DOCUMENT: META 10-K 2022


In [6]:
print(f"len(document): {len(document)}")
print(f"type(document): {type(document)}")
print(f"type(document[0]): {type(document[0])}")

len(document): 1
type(document): <class 'list'>
type(document[0]): <class 'llama_index.schema.Document'>


In [7]:
import anyio
from app.chat.messaging import ChatCallbackHandler

send_chan, recv_chan = anyio.create_memory_object_stream(100)
callback_handler = ChatCallbackHandler(send_chan)

In [8]:
from app.chat.engine import get_tool_service_context
original_service_context = get_tool_service_context(callback_handlers=[callback_handler], node_parser_type="original")

######################### CREATING SERVICE CONTEXT USING original NODE PARSER #########################


Parse Nodes

In [9]:
# display original node parsing
original_node_parser = original_service_context.node_parser
original_nodes = original_node_parser.get_nodes_from_documents(document)
print(f"Total nodes: {len(original_nodes)}")

from eval import format_pdf_text
print(f"\n{'#'*50} ORIGINAL NODE {'#'*50}\n{format_pdf_text(original_nodes[5].text)}")

Total nodes: 327

################################################## ORIGINAL NODE ##################################################
We have based these forward-looking statements largely on our current expectations and projections about future events and trends that we believe may affect our financial condition, results of operations, business strategy, short-term and long-term business operations and objectives, and financial needs. These forward-looking statements are subject to a number of risks, uncertainties and assumptions, including those described in Part I, Item 1A, "Risk Factors" in this Annual Report on Form 10-K. Moreover, we operate in a very competitive and rapidly changing environment. New risks emerge from time to time. It is not possible for our management to predict all risks, nor can we assess the impact of all factors on our business or the extent to which any factor, or combination of factors, may cause actual results to differ materially from those contained in 

In [10]:
sentence_window_service_context = get_tool_service_context(callback_handlers=[callback_handler], node_parser_type="sentence-window")
sentence_window_node_parser = sentence_window_service_context.node_parser
sentence_window_nodes = sentence_window_node_parser.get_nodes_from_documents(document)

print(f"Total sentence-window nodes: {len(sentence_window_nodes)}")
sentence = format_pdf_text(sentence_window_nodes[5].metadata.get("original_text"))
window = format_pdf_text(sentence_window_nodes[5].metadata.get("window"))

print(f"window_size = {sentence_window_node_parser.window_size}")

print(f"\nSentence-Window node:")
print(f"\n{'#'*50} SENTENCE {'#'*50}\n{sentence}")
print(f"\n{'#'*50} WINDOW {'#'*50}\n{window}")


######################### CREATING SERVICE CONTEXT USING sentence-window NODE PARSER #########################
Total sentence-window nodes: 2184
window_size = 2

Sentence-Window node:

################################################## SENTENCE ##################################################
Yes ☐ No ☒ Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 (Exchange Act) during the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days.

################################################## WINDOW ##################################################
Employer Identification Number) 1601 Willow Road , Menlo Park , California 94025 (Address of principal executive offices and Zip Code) ( 650 ) 543-4800 (Registrant's telephone number, including area code) __________________

In [11]:
# parse nodes hierarchically
from llama_index.node_parser import get_leaf_nodes, get_root_nodes
auto_merging_service_context = get_tool_service_context(callback_handlers=[callback_handler], node_parser_type="hierarchical")

hierarchical_node_parser = auto_merging_service_context.node_parser
hierarchical_nodes = hierarchical_node_parser.get_nodes_from_documents(document)
leaf_nodes = get_leaf_nodes(hierarchical_nodes)
root_nodes = get_root_nodes(hierarchical_nodes)

######################### CREATING SERVICE CONTEXT USING hierarchical NODE PARSER #########################


In [12]:
print(f"Node hierarchy (chunk sizes): {hierarchical_node_parser.chunk_sizes}")
print(f"Total hierarchical nodes: {len(hierarchical_nodes)}")
print(f"Total leaf nodes: {len(leaf_nodes)}")

Node hierarchy (chunk sizes): [2048, 512, 128]
Total hierarchical nodes: 2556
Total leaf nodes: 2119


In [13]:
# function to get parent of a hierarchical node
get_parent_node = lambda node, all_nodes: next(i for i in all_nodes if i.id_ == node.parent_node.node_id)

# get intermediate & root nodes
leaf_node = leaf_nodes[0]
intermediate_node = get_parent_node(leaf_node, hierarchical_nodes)
root_node = get_parent_node(intermediate_node, hierarchical_nodes)

print(f"Notice how each node is a subset of its parent:")
print(f"\n{'#'*50} LEAF NODE {'#'*50}\n{format_pdf_text(leaf_node.text)}")
print(f"\n{'#'*50} INTERMEDIATE NODE {'#'*50}\n{format_pdf_text(intermediate_node.text)}")
print(f"\n{'#'*50} ROOT NODE {'#'*50}\n{format_pdf_text(root_node.text)}")


Notice how each node is a subset of its parent:

################################################## LEAF NODE ##################################################
UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C.

################################################## INTERMEDIATE NODE ##################################################
UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 __________________________ FORM 10-K __________________________ (Mark One) ☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31 , 2022 or ☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from to Commission File Number: 001-35551 __________________________ Meta Platforms, Inc. (Exact name of registrant as specified in its charter) __________________________ Delaware 20-1665019 (State or other jurisdiction of incorporation or organization

Build Indexes

In [14]:
import os
from llama_index.indices.vector_store.base import VectorStoreIndex
from llama_index import StorageContext, load_index_from_storage

ORIGINAL_PERSIST_DIR = '/workspaces/sec-insights/backend/eval/index_storage/original'   # local dir to persist storage of index
if not os.path.exists(ORIGINAL_PERSIST_DIR):                                            # check if storage already exists
    print(f"Creating Original index and saving it at: {ORIGINAL_PERSIST_DIR}")
    original_index = VectorStoreIndex(original_nodes)                                   # create the index
    original_index.storage_context.persist(persist_dir=ORIGINAL_PERSIST_DIR)            # store it for later
else:
    print(f"Original index exists - loading it from: {ORIGINAL_PERSIST_DIR}")
    original_storage_context = StorageContext.from_defaults(persist_dir=ORIGINAL_PERSIST_DIR)   # load the existing index
    original_index = load_index_from_storage(original_storage_context)

SENTENCE_WINDOW_PERSIST_DIR = '/workspaces/sec-insights/backend/eval/index_storage/setence_window'
if not os.path.exists(SENTENCE_WINDOW_PERSIST_DIR):
    print(f"Creating Sentence-Window index and saving it at: {SENTENCE_WINDOW_PERSIST_DIR}")
    sentence_window_index = VectorStoreIndex.from_documents(
        document,
        service_context=sentence_window_service_context,
    )
    sentence_window_index.storage_context.persist(persist_dir=SENTENCE_WINDOW_PERSIST_DIR)
else:
    print(f"Sentence-Window index exists - loading it from: {SENTENCE_WINDOW_PERSIST_DIR}")
    setence_window_storage_context = StorageContext.from_defaults(persist_dir=SENTENCE_WINDOW_PERSIST_DIR)
    sentence_window_index = load_index_from_storage(
        storage_context=setence_window_storage_context,
        service_context=sentence_window_service_context,
    )

AUTO_MERGING_PERSIST_DIR = '/workspaces/sec-insights/backend/eval/index_storage/auto_merging'
auto_merging_storage_context = StorageContext.from_defaults()
auto_merging_storage_context.docstore.add_documents(hierarchical_nodes)
if not os.path.exists(AUTO_MERGING_PERSIST_DIR):
    print(f"Creating Auto-Merging index and saving it at: {AUTO_MERGING_PERSIST_DIR}")
    auto_merging_index = VectorStoreIndex(
        leaf_nodes,
        storage_context=auto_merging_storage_context,
        service_context=auto_merging_service_context
    )
    auto_merging_index.storage_context.persist(AUTO_MERGING_PERSIST_DIR)
else:
    print(f"Auto-Merging index exists - loading it from: {AUTO_MERGING_PERSIST_DIR}")
    auto_merging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir=AUTO_MERGING_PERSIST_DIR),
        service_context=auto_merging_service_context
    )

Original index exists - loading it from: /workspaces/sec-insights/backend/eval/index_storage/original
Sentence-Window index exists - loading it from: /workspaces/sec-insights/backend/eval/index_storage/setence_window
Auto-Merging index exists - loading it from: /workspaces/sec-insights/backend/eval/index_storage/auto_merging


Build Query Engines

In [15]:
# build original query engine
original_query_engine = original_index.as_query_engine(
    similarity_top_k=3                                      # same as original source code
)

In [16]:
# build sentence-window query engine
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor, LLMRerank
postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
sentence_window_rerank = LLMRerank(
    top_n=2,
    service_context=sentence_window_service_context,
)
sentence_window_query_engine = sentence_window_index.as_query_engine(
    similarity_top_k=4, node_postprocessors=[postproc, sentence_window_rerank]
)

In [17]:
# build auto-merging query engine
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine
base_retriever = auto_merging_index.as_retriever(
    similarity_top_k=12,
)
auto_merging_retriever = AutoMergingRetriever(
    base_retriever, auto_merging_storage_context,
)
auto_merging_rerank = LLMRerank(
    top_n=4,
    service_context=auto_merging_service_context,
)
auto_merging_query_engine = RetrieverQueryEngine.from_args(
    auto_merging_retriever, node_postprocessors=[auto_merging_rerank]
)

#### Compare The Responses of each Query Engine

In [22]:
prompt = "What is Meta's mission?"
print(f"Prompt: {prompt}")
print(f"Excerpt from META 2022 10-K Document: Our mission is to give people the power to build community and bring the world closer together.")

# original_response = original_query_engine.query(prompt)
# sentence_window_response = sentence_window_query_engine.query(prompt)
auto_merging_response = auto_merging_query_engine.query(prompt)

# print(f"\nORIGINAL QUERY ENGINE RESPONSE:\n{str(original_response)}")
# print(f"\nSENTENCE-WINDOW QUERY ENGINE RESPONSE:\n{str(sentence_window_response)}")
print(f"\nAUTO-MERGING QUERY ENGINE RESPONSE:\n{str(auto_merging_response)}")

Prompt: What is Meta's mission?
Excerpt from META 2022 10-K Document: Our mission is to give people the power to build community and bring the world closer together.


ValueError: doc_id 8c0074b7-64f9-4402-b271-9ffbd54c6a70 not found.

Generate Evaluation Dataset

In [ ]:
import asyncio
import nest_asyncio
nest_asyncio.apply()    # allow asynchonous code to run within Jupyter notebook

In [ ]:
from eval import generate_dataset
from llama_index.node_parser import SentenceSplitter
from llama_index.evaluation import DatasetGenerator, QueryResponseDataset
from llama_index.llms.openai import OpenAI
from llama_index import ServiceContext
import random

file_path="/workspaces/sec-insights/backend/eval/eval_dataset.json"     # path to save evaluation dataset
if not os.path.exists(file_path):
    text_splitter = SentenceSplitter()
    base_nodes = text_splitter.get_nodes_from_documents(document)

    # Use the middle 80% of document context to generate questions in evaluation dataset
    start_index = int(len(base_nodes) * 0.1)
    end_index = int(len(base_nodes) * 0.9)

    num_nodes_eval = 30    #  The number of nodes (randomly sampled from total nodes) to use for generating evaluation questions.
    sample_eval_nodes = random.sample(base_nodes[start_index:end_index], num_nodes_eval)

    dataset_generator_llm = OpenAI(temperature=0, model="gpt-4")
    dataset_generator_service_context = ServiceContext.from_defaults(llm=dataset_generator_llm)
    
    dataset_generator = DatasetGenerator(
        nodes=sample_eval_nodes,
        # service_context=original_service_context,
        # service_context=dataset_generator_service_context,        # rate limiting issues using gpt-4
        num_questions_per_chunk=2,
        show_progress=True,
    )
    eval_dataset = await dataset_generator.agenerate_dataset_from_nodes()
    eval_dataset.save_json(file_path)
    print(f"Saved evaluation dataset at: {file_path}")

    # dataset_generator_SW_service_context = DatasetGenerator(
    #     nodes=sample_eval_nodes,
    #     service_context=sentence_window_service_context,
    #     num_questions_per_chunk=2,
    #     show_progress=True,
    # )
    # eval_dataset_SW_service_context = await dataset_generator_SW_service_context.agenerate_dataset_from_nodes()
    # eval_dataset_SW_service_context.save_json("/workspaces/sec-insights/backend/eval/eval_dataset_SW_service_context.json")
    # print(f"Saved evaluation dataset at: /workspaces/sec-insights/backend/eval/eval_dataset_SW_service_context.json")

else: 
    print(f"Evaluation dataset already exists at: {file_path}")
    eval_dataset = QueryResponseDataset.from_json(file_path)


Evaluate Query Engines

In [ ]:
_='''
Note - The following code snippet had to be added to the CorrectnessEvaluator class within the llama-index v"0.9.7" package at
llama_index/evaluation/correctness.py before line: score_str, reasoning_str = eval_response.split("\n", 1)
This resolves an issue where eval_resonse is created beginning with a newline character, resulting in an error trying to convert
an empty str to a float on line: score = float(score_str).

Code snippet:
if eval_response[0] == '\n':
    eval_response = eval_response[1:]
'''

In [ ]:
'''
Evaluate responses based on the following metrics:
    Correctness:            The correctness of a response - A score between 1 (worst) and 5 (best).
    Semantic Similarity:    The similarity between embeddings of the generated answer and reference answer.
    Relevance:              The relevance of retrieved context and response to the query. Considers the query string, retrieved context, and response string.
    Faithfulness:           How well the response is supported by the retrieved context (i.e., Is there hallucination?)
'''
from llama_index.evaluation import CorrectnessEvaluator, SemanticSimilarityEvaluator, RelevancyEvaluator, FaithfulnessEvaluator, BatchEvalRunner
from llama_index.evaluation.eval_utils import get_responses

evaluator_c = CorrectnessEvaluator()
evaluator_s = SemanticSimilarityEvaluator()
evaluator_r = RelevancyEvaluator()
evaluator_f = FaithfulnessEvaluator()

evaluator_dict = {
    "correctness": evaluator_c,
    "faithfulness": evaluator_f,
    "relevancy": evaluator_r,
    "semantic_similarity": evaluator_s,
}
batch_runner = BatchEvalRunner(evaluator_dict, workers=2, show_progress=True)

eval_qs = eval_dataset.questions
ref_response_strs = [r for (_, r) in eval_dataset.qr_pairs]
print(f"{len(eval_qs)} questions in dataset")

Get Responses from Each Query Engine for Evaluation

In [ ]:
max_samples = 35

import time
time.sleep(12)
print(f"Original Query Engine")
original_pred_responses = get_responses(
    eval_qs[:max_samples], original_query_engine, show_progress=True
)
time.sleep(12)
print(f"Sentence-Window Query Engine")
sentence_window_pred_responses = get_responses(
    eval_qs[:max_samples], sentence_window_query_engine, show_progress=True
)
time.sleep(12)
print(f"Auto-Merging Query Engine")
auto_merging_pred_responses = get_responses(
    eval_qs[:max_samples], auto_merging_query_engine, show_progress=True
)

In [ ]:
max_samples = 20

In [ ]:
print(f"Original Query Engine")
original_pred_responses = get_responses(
    eval_qs[:max_samples], original_query_engine, show_progress=True
)

In [ ]:
print(f"Sentence-Window Query Engine")
sentence_window_pred_responses = get_responses(
    eval_qs[:max_samples], sentence_window_query_engine, show_progress=True
)

In [ ]:
print(f"Auto-Merging Query Engine")
auto_merging_pred_responses = get_responses(
    eval_qs[:max_samples], auto_merging_query_engine, show_progress=True
)

Evaluate Responses from Each Query Engine

In [ ]:
print(f"Original Query Engine")
original_eval_results = await batch_runner.aevaluate_responses(
    queries=eval_qs[:max_samples],
    responses=original_pred_responses[:max_samples],
    reference=ref_response_strs[:max_samples],
)
print(f"Sentence-Window Query Engine")
sentence_window_eval_results = await batch_runner.aevaluate_responses(
    queries=eval_qs[:max_samples],
    responses=sentence_window_pred_responses[:max_samples],
    reference=ref_response_strs[:max_samples],
)
print(f"Auto-Merging Query Engine")
auto_merging_eval_results = await batch_runner.aevaluate_responses(
    queries=eval_qs[:max_samples],
    responses=auto_merging_pred_responses[:max_samples],
    reference=ref_response_strs[:max_samples],
)

Get Evaluation Results

In [ ]:
from llama_index.evaluation.eval_utils import get_results_df
results_df = get_results_df(
    [original_eval_results, sentence_window_eval_results, auto_merging_eval_results],
    ["Base Retriever", "Sentence-Window Retriever", "Auto-Merging Retriever"],
    ["correctness", "relevancy", "faithfulness", "semantic_similarity"],
)
display(results_df)

In [ ]:
# save results to CSV file
OUTPUT_PATH = '/workspaces/sec-insights/backend/eval/results.csv'
results_df.to_csv(OUTPUT_PATH, index=False)

In [ ]:
# from llama_index.node_parser import SentenceWindowNodeParser
# from llama_index.callbacks.base import BaseCallbackHandler, CallbackManager
# callback_manager = CallbackManager([callback_handler])
# sw_parser = SentenceWindowNodeParser.from_defaults(
#     window_size=3,
#     window_metadata_key="window",
#     original_text_metadata_key="original_text",
#     callback_manager=callback_manager,
# )
# llm = OpenAI(
#     temperature=0,
#     model="gpt-3.5-turbo-0613",
#     streaming=False,
# )
# from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingMode, OpenAIEmbeddingModelType
# embedding_model = OpenAIEmbedding(
#     mode=OpenAIEmbeddingMode.SIMILARITY_MODE,
#     model_type=OpenAIEmbeddingModelType.TEXT_EMBED_ADA_002,
# )
# sw_service_context = ServiceContext.from_defaults(
#     callback_manager=[callback_handler],
#     llm=llm,
#     embed_model=embedding_model,
#     node_parser=sw_parser,
# )
# sw_index = VectorStoreIndex.from_documents(
#     document,
#     service_context=sw_service_context,
# )
# sw_postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
# sw_rerank = LLMRerank(
#     top_n=4,
#     service_context=sw_service_context,
# )
# sw_query_engine = sw_index.as_query_engine(
#     similarity_top_k=8, node_postprocessors=[sw_postproc, sw_rerank]
# )

